In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pickle
with open('/kaggle/input/adni-sorted/pixels_CN_1.pkl', 'rb') as f:
    cn_1 = pickle.load(f)
with open('/kaggle/input/adni-sorted/pixels_CN_2.pkl', 'rb') as f:
    cn_2 = pickle.load(f)
with open('/kaggle/input/adni-sorted/pixels_MCI.pkl', 'rb') as f:
    mci = pickle.load(f)
    
cn_1.extend(cn_2)

mci.pop(70)
del cn_2
print('dataset loaded')
## Change the shape of patient 2, slice 6 in MCI

dataset loaded


In [4]:
import random

# Set the seed
random.seed(42)
# Calculate the number of samples
l_cn = len(cn_1)
l_mci = len(mci)
num_samples_1 = int(0.8 * l_cn)
num_samples_2 = int(0.8* l_mci)

train_indices_1 = random.sample(list(range(l_cn)), num_samples_1)
train_indices_2 = random.sample(list(range(l_mci)), num_samples_2)

test_indices_1 = list(set(list(range(l_cn))) - set(train_indices_1))
test_indices_2 = list(set(list(range(l_mci))) - set(train_indices_2))

def scale(arr):
    return (arr - np.min(arr))/(np.max(arr) - np.min(arr))

# # Sample 80% of the entries
# X_train_1 = [np.array([scale(cn_1[i][j]) for j in range(65)]) for i in train_indices_1]
# np.save('X_train_1.npy',np.array(X_train_1))
# X_train_2 = [np.array([scale(mci[i][j]) for j in range(65)]) for i in train_indices_2]
# np.save('X_train_2.npy', np.array(X_train_2))

# del X_train_1, X_train_2, train_indices_1, train_indices_2


In [5]:
X_test_1 = [np.array([scale(cn_1[i][j]) for j in range(65)]) for i in test_indices_1]
np.save('X_test_1.npy',np.array(X_test_1))
X_test_2 = [np.array([scale(mci[i][j]) for j in range(65)]) for i in test_indices_2]
np.save('X_test_2.npy',np.array(X_test_2))

del X_test_1, X_test_2, test_indices_1, test_indices_2
#del mci, cn_1

In [2]:
X_train_1 = np.load('/kaggle/input/x-train-1-2/X_train_1.npy')
X_train_2 = np.load('/kaggle/input/x-train-1-2/X_train_2.npy')

# Create labels for your data
y_train_1 = np.zeros(len(X_train_1))  # Class 0 for X_train_1
y_train_2 = np.ones(len(X_train_2))   # Class 1 for X_train_2

# Stack your data and labels
X_train = np.vstack((X_train_1, X_train_2))
y_train = np.hstack((y_train_1, y_train_2))

In [3]:
del X_train_1, X_train_2, y_train_1, y_train_2

In [4]:
from sklearn.utils import shuffle

# Shuffle your data and labels
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Now, X_train is your shuffled data and y_train are the corresponding labels

In [5]:
np.save('y_train.npy', y_train)
del y_train
np.save('X_train.npy', X_train)
del X_train


In [8]:
X_test_1 = np.load('/kaggle/working/X_test_1.npy')
X_test_2 = np.load('/kaggle/working/X_test_2.npy')

# # Convert your lists to numpy arrays
# X_test_1 = np.stack(X_test_1)
# X_test_2 = np.stack(X_test_2)

# Create labels for your data
y_test_1 = np.zeros(len(X_test_1))  # Class 0 for X_train_1
y_test_2 = np.ones(len(X_test_2))   # Class 1 for X_train_2

# Stack your data and labels
X_test = np.vstack((X_test_1, X_test_2))
y_test = np.hstack((y_test_1, y_test_2))

from sklearn.utils import shuffle
# Shuffle your data and labels
X_test, y_test = shuffle(X_test, y_test, random_state=42)
np.save('X_test.npy', X_test)
del X_test
np.save('y_test.npy', y_test)
del y_test

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader #, TensorDataset
import torch.nn.functional as F

from torch.utils.data import Dataset

X_train = np.load('/kaggle/input/x-train-1-2/X_train.npy')
y_train = np.load('/kaggle/input/x-train-1-2/y_train.npy')


# Define your custom dataset
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Create dataset
train_dataset = MyDataset(X_train, y_train)


In [236]:
# Create DataLoaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(65, 4, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256*240, 60)  
        self.fc3 = nn.Linear(60, 2)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer

    def forward(self, x):
        x = x.view(-1, 65, 256, 240)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = x.view(x.size(0), -1)  # Flatten layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Initialize the network and print its architecture
model = Net()
print(model)

Net(
  (conv1): Conv2d(65, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=61440, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [237]:
model = Net()
# Use CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
model.to(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight = torch.tensor([1.0,2.0]).to(device))
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# Initialize empty lists to store losses
train_losses = []

# Training loop
for epoch in range(15):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels_onehot = torch.zeros(labels.size(0), 2, device=device)
        labels = labels.long()
        labels_onehot.scatter_(1, labels.view(-1, 1), 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        l1_lambda = 0.001
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = criterion(outputs, labels_onehot)
        loss+=l1_lambda*l1_norm
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

print('Finished Training')

Epoch 1, training loss: 3.5320
Epoch 2, training loss: 1.1930
Epoch 3, training loss: 1.0866
Epoch 4, training loss: 1.0360
Epoch 5, training loss: 1.0347
Epoch 6, training loss: 1.0290
Epoch 7, training loss: 1.0687
Epoch 8, training loss: 1.0334
Epoch 9, training loss: 1.0184
Epoch 10, training loss: 1.0544
Epoch 11, training loss: 1.0440
Epoch 12, training loss: 1.0332
Epoch 13, training loss: 1.0375
Epoch 14, training loss: 1.0309
Epoch 15, training loss: 1.0460
Finished Training


In [238]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop without regularization
for epoch in range(15):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels_onehot = torch.zeros(labels.size(0), 2, device=device)
        labels = labels.long()
        labels_onehot.scatter_(1, labels.view(-1, 1), 1)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels_onehot)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

print('Finished Training')

Epoch 1, training loss: 0.8937
Epoch 2, training loss: 0.8591
Epoch 3, training loss: 0.8144
Epoch 4, training loss: 0.7782
Epoch 5, training loss: 0.7312
Epoch 6, training loss: 0.6815
Epoch 7, training loss: 0.6078
Epoch 8, training loss: 0.5574
Epoch 9, training loss: 0.5251
Epoch 10, training loss: 0.4513
Epoch 11, training loss: 0.3829
Epoch 12, training loss: 0.3827
Epoch 13, training loss: 0.3351
Epoch 14, training loss: 0.3174
Epoch 15, training loss: 0.2225
Finished Training


In [5]:
# # del X_train, y_train
# X_test = np.load('/kaggle/input/x-train-1-2/X_test.npy')
# y_test = np.load('/kaggle/input/x-train-1-2/y_test.npy')

In [239]:

test_dataset = MyDataset(X_test, y_test)
from sklearn.metrics import confusion_matrix
import torch
import numpy as np

# Assuming x_test is a PyTorch tensor
testloader = torch.utils.data.DataLoader(X_test, batch_size=8)

model.eval()  # Set the model to evaluation mode
predictions = []
output = []
with torch.no_grad():
    for data in testloader:
        outputs = model(data.float().to(device)).to(device)
        predicted = torch.softmax(outputs, dim = 1) # Apply a threshold
        predictions.extend(predicted.tolist())
        output.extend(outputs.tolist())


def find_optimal_threshold(predictions, y_test):
    min_sum = float('inf')
    optimal_threshold = 0

    # Iterate over possible thresholds from 0 to 1
    for threshold in np.arange(0.0, 1.01, 0.001):
        # Apply threshold
        preds = (np.array(predictions)[:,1] > threshold).astype(int)

        # Compute confusion matrix
        cm = confusion_matrix(y_test, preds)

        # Compute sum of off-diagonal elements
        off_diagonal_sum = 164 - np.trace(cm)
        #print(cm)
        # Update optimal threshold if this threshold is better
        if off_diagonal_sum < min_sum and cm[1][1]/np.sum(cm[1]) >0.5:
            min_sum = off_diagonal_sum
            optimal_threshold = threshold

    return optimal_threshold

# Assuming 'predictions' are your predicted probabilities and 'y_test' are your true labels
optimal_threshold = find_optimal_threshold(predictions, y_test)        
# Calculate the multi-label confusion matrix
mcm = confusion_matrix(y_test, (np.array(predictions)[:,1]>optimal_threshold).astype(int))

print('Confusion Matrix:')
for i, matrix in enumerate(mcm):
    print(f'Class {i}:')
    print(matrix)


Confusion Matrix:
Class 0:
[44 38]
Class 1:
[18 19]


In [240]:
predictions

[[0.7328335046768188, 0.26716652512550354],
 [0.02444249764084816, 0.9755574464797974],
 [0.8224502801895142, 0.17754971981048584],
 [0.9875355362892151, 0.01246438268572092],
 [0.9425466060638428, 0.05745336413383484],
 [0.9286924004554749, 0.07130758464336395],
 [0.839780330657959, 0.16021963953971863],
 [0.9614708423614502, 0.0385291762650013],
 [0.9723830819129944, 0.027616865932941437],
 [0.8485738039016724, 0.15142613649368286],
 [0.9778349995613098, 0.02216499298810959],
 [0.9790586829185486, 0.020941313356161118],
 [0.9510384202003479, 0.048961520195007324],
 [0.6394004821777344, 0.3605995774269104],
 [0.879150390625, 0.120849609375],
 [0.9772056341171265, 0.022794289514422417],
 [0.8269427418708801, 0.17305722832679749],
 [0.009851256385445595, 0.990148663520813],
 [0.9271709322929382, 0.07282905280590057],
 [0.9448071122169495, 0.055192895233631134],
 [0.529174268245697, 0.47082576155662537],
 [0.8428623080253601, 0.15713773667812347],
 [0.9924888610839844, 0.0075111929327249

In [220]:
y_test

array([0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1.])

In [241]:
# Simpler model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(65, 2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128*240, 30)  
        self.fc3 = nn.Linear(30, 2)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer

    def forward(self, x):
        x = x.view(-1, 65, 256, 240)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = x.view(x.size(0), -1)  # Flatten layer
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
# Initialize the network and print its architecture
model = Net()
print(model)

Net(
  (conv1): Conv2d(65, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=30720, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [247]:
# Use CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
model.to(device)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight = torch.tensor([1.0,3.0]).to(device))
optimizer = optim.Adam(model.parameters(), lr=1e-4)


# Initialize empty lists to store losses
train_losses = []

# Training loop
for epoch in range(15):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels_onehot = torch.zeros(labels.size(0), 2, device=device)
        labels = labels.long()
        labels_onehot.scatter_(1, labels.view(-1, 1), 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        l1_lambda = 0.001
        l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = criterion(outputs, labels_onehot)
        loss+=l1_lambda*l1_norm
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

print('Finished Training')

Epoch 1, training loss: 1.9654
Epoch 2, training loss: 1.4470
Epoch 3, training loss: 1.2368
Epoch 4, training loss: 1.1353
Epoch 5, training loss: 1.0957
Epoch 6, training loss: 1.0755
Epoch 7, training loss: 1.0677
Epoch 8, training loss: 0.9710
Epoch 9, training loss: 1.0508
Epoch 10, training loss: 0.9714
Epoch 11, training loss: 1.0511
Epoch 12, training loss: 0.9834
Epoch 13, training loss: 0.9784
Epoch 14, training loss: 0.9487
Epoch 15, training loss: 0.9369
Finished Training


In [248]:
### optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(15):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.float()
        labels_onehot = torch.zeros(labels.size(0), 2, device=device)
        labels = labels.long()
        labels_onehot.scatter_(1, labels.view(-1, 1), 1)

        optimizer.zero_grad()
        outputs = model(inputs)
        
        #l1_lambda = 0.001
        #l1_norm = sum(p.abs().sum() for p in model.parameters())
        loss = criterion(outputs, labels_onehot)
        #loss+=l1_lambda*l1_norm
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Compute training loss
        train_losses.append(loss.item())
    print(f'Epoch {epoch+1}, training loss: {running_loss/len(train_dataloader):.4f}')

print('Finished Training')

Epoch 1, training loss: 0.3254
Epoch 2, training loss: 0.2787
Epoch 3, training loss: 0.2563
Epoch 4, training loss: 0.1999
Epoch 5, training loss: 0.2448
Epoch 6, training loss: 0.2003
Epoch 7, training loss: 0.1866
Epoch 8, training loss: 0.1581
Epoch 9, training loss: 0.1975
Epoch 10, training loss: 0.1601
Epoch 11, training loss: 0.1733
Epoch 12, training loss: 0.1513
Epoch 13, training loss: 0.1048
Epoch 14, training loss: 0.1637
Epoch 15, training loss: 0.1306
Finished Training


In [249]:

test_dataset = MyDataset(X_test, y_test)
from sklearn.metrics import confusion_matrix

# Assuming x_test is a PyTorch tensor
testloader = torch.utils.data.DataLoader(X_test, batch_size=8)

model.eval()  # Set the model to evaluation mode
predictions = []
output = []
with torch.no_grad():
    for data in testloader:
        outputs = model(data.float().to(device)).to(device)
        predicted = torch.softmax(outputs, dim = 1) # Apply a threshold
        predictions.extend(predicted.tolist())
        output.extend(outputs.tolist())

        
optimal_threshold = find_optimal_threshold(predictions, y_test)
# Calculate the multi-label confusion matrix
mcm = confusion_matrix(y_test, (np.array(predictions)[:,1]>optimal_threshold).astype(int))

print('Confusion Matrix:')
for i, matrix in enumerate(mcm):
    print(f'Class {i}:')
    print(matrix)

    


Confusion Matrix:
Class 0:
[53 29]
Class 1:
[17 20]
